<a href="https://colab.research.google.com/github/yoonkim313/dataCampusProject-Team10/blob/master/Relation%20Extraction/AutomatedRelationExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!pip install transformers
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy
!pip install kss
%cd /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
!pip install python-pptx
from frameBERT import frame_parser
path="/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 890kB 12.4MB/s 
     |████████████████████████████████| 3.0MB 23.5MB/s 
     |████████████████████████████████| 1.1MB 29.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=00e03b42e3ddae0e8fcd988ac4b4bac61009becae1e0ef3c448d01a01aeb1283
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a5012363

In [5]:
!pip install python-pptx

In [19]:
import kss
from konlpy.tag import Hannanum
h = Hannanum()
import re
import time
import numpy as np
import heapq
from pprint import pprint
import pandas as pd
from operator import itemgetter 
from pptx import Presentation
from pptx.util import Inches
from pptx.enum.shapes import MSO_SHAPE
from collections import deque
from ast import literal_eval
from collections import defaultdict

In [98]:
text ='''
신체의 세포, 조직, 장기가 손상되어 더 이상 제 기능을 하지못할 때에 이를 대체하기 위해 이식을 실시한다. 
이때 이식으로 옮겨 붙이는 세포, 조직, 장기를 이식편이라 한다. 자신이나 일란성 쌍둥이의 이식편을 이용할 
수 없다면 다른 사람의 이식편으로 ‘동종 이식’을 실시한다. 
'''
text = re.sub("\n"," ",text)
pprint(text)
t = '''
그런데 우리의 몸은 자신의 것이 아닌 물질이 
체내로 유입될 경우 면역 반응을 일으키므로, 유전적
으로 동일하지 않은 이식편에 대해 항상 거부 반응을 일으킨다.
면역적 거부 반응은 면역 세포가 표면에 발현하는 주조직적합
복합체(MHC) 분자의 차이에 의해 유발된다. 개체마다 MHC에
차이가 있는데 서로 간의 유전적 거리가 멀수록 MHC에 차이가
커져 거부 반응이 강해진다. 이를 막기 위해 면역 억제제를 사용
하는데, 이는 면역 반응을 억제하여 질병 감염의 위험성을 높인다.
이식에는 많은 비용이 소요될 뿐만 아니라 이식이 가능한 동종
이식편의 수가 매우 부족하기 때문에 이를 대체하는 방법이 개발
되고 있다. 우선 인공 심장과 같은 ‘전자 기기 인공 장기’를 이용
하는 방법이 있다. 하지만 이는 장기의 기능을 일시적으로 대체
하는 데 사용되며, 추가 전력 공급 및 정기적 부품 교체 등이
요구되는 단점이 있고, 아직 인간의 장기를 완전히 대체할 만큼
정교한 단계에 이르지는 못했다.
'''

(' 신체의 세포, 조직, 장기가 손상되어 더 이상 제 기능을 하지못할 때에 이를 대체하기 위해 이식을 실시한다.  이때 이식으로 옮겨 '
 '붙이는 세포, 조직, 장기를 이식편이라 한다. 자신이나 일란성 쌍둥이의 이식편을 이용할  수 없다면 다른 사람의 이식편으로 ‘동종 '
 '이식’을 실시한다.  ')


In [91]:
parser = frame_parser.FrameParser(model_path=path, language='ko')

srl model: framenet
language: ko
version: 1.2
using viterbi: False
using masking: True
pretrained BERT: bert-base-multilingual-cased
using TGT special token: True
used dictionary:
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lu2idx.json
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lufrmap.json
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/mul_bio_frargmap.json
...loaded model path: /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
...model is loaded


### Linked List Implemented to Show the whole list of sentences and their parsed tags in a consecutive manner

In [99]:
def split_into_sentences(object):
  ls = kss.split_sentences(object)
  return ls

def frameParse(text):
  parser = frame_parser.FrameParser(model_path=path, language='ko')
  parsed = parser.parser(text, sent_id='1', result_format='conll')
  return parsed
  

def sortCandidate(parsed, num_candidates):
  for i in range(len(parsed)):
    count=0
    for element in parsed[i][3]:
      if element.startswith("O"):
        count+=1
  q[i] = len(words) - count
  hq = heapq.nlargest(num_candidates, q)
  indices.append(q.index(hq[idx]))
  # for idx in range(num_candidates):
  #   temp = parsed[q.index(hq[idx])]
  #   words = temp[0]
  #   [[row[i] for row in temp] for i in range(len(words))]
  return indices


def findConsecutiveBIO(words, tag):
  began = False
  count = 0
  q = deque(words)
  for i in range(len(words)):
    if tag[i]=='O':
      q.popleft()
    if tag[i].startswith('B'):
      began=True
    if began & tag[i].startswith('I'):
      count+=1
  return q

# Linked List 구현해서 parsed 후보군에 사용
class Node:
    def __init__(self, data):
        self.words = data[0]
        self.tags = data[3]
        self.next = None

    def __repr__(self):
        return str((self.words, self.tags))

class LinkedList:
  def __init__(self):
      self.head = None

  def __repr__(self):
      node = self.head
      nodes = []
      while node is not None:
          nodes.append(str(node.tags))
          node = node.next
      nodes.append("None")
      return ' -> '.join(nodes)
  
class LinkedList:
  def __init__(self):
      self.head = None

  def __repr__(self):
      node = self.head
      nodes = []
      while node is not None:
          nodes.append(str(node.tags))
          node = node.next
      nodes.append("None")
      return ' -> '.join(nodes)

def extractFrame(text):
  ls = split_into_sentences(text)
  final = {}
  
  for idx in range(len(ls)):
    parsed = frameParse(ls[idx]) # candidates 생성
    final.setdefault(idx,str)
    parsedList =  LinkedList()
    for j in range(len(parsed)):
      parsed_candidate = parsed[j]
      new_node = Node(parsed_candidate)
      if j == 0:
        old_node = new_node
        parsedList.head = old_node
      elif j==len(parsed)-1:
        old_node.next = new_node
        new_node.next = None
        print(idx,'  ',parsedList)
        final[idx] = parsedList
      else:
        old_node.next = new_node
        old_node = new_node
  return final



In [125]:
def main(text):
  all = extractFrame(text)
  s = ""
  res = {}

  i = 1
  res.setdefault(i,tuple)
  for k, v in all.items():
    a = v.head
    while a is not None:
      temp = " "
      if len(a.words) == len(a.tags):
        q = findConsecutiveBIO(a.words, a.tags)
      else:
        print("ERROR OCCURED")
        
      count = len(a.words)
      for tag in a.tags:
       if tag.startswith("O"):
          count -= 1
      if count > len(a.words)*0.5:
        print("The threshold is reached !! 😛 😝 😜 🤪 🤨 🧐 🤓 😎 🤩 🥳 😏")
        temp = " ".join(q) 
        print(temp)
        words = a.words
        tags = a.tags
      res[i] = (a.words, a.tags , " ".join(q))
      a = a.next
    i+=1
  return res


In [126]:
extracted = main(text)
pprint(extracted)

srl model: framenet
language: ko
version: 1.2
using viterbi: False
using masking: True
pretrained BERT: bert-base-multilingual-cased
using TGT special token: True
used dictionary:
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lu2idx.json
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lufrmap.json
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/mul_bio_frargmap.json
...loaded model path: /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
...model is loaded


/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/utils.py:279: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


0    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] -> ['O', 'O', 'O', 'O', 'O', 'O', 'I-Class', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] -> ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] -> ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Time', 'O', 'O', 'O', 'O', 'O'] -> ['O', 'O', 'O', 'I-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'O', 'O', 'I-Trajector_event', 'I-Trajector_event', 'O', 'I-Trajector_event'] -> ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] -> ['O', 'O', 'O', 'I-Time', 'I-Time', 'I-Time', 'I-Time', 'I-Time', 'I-Time', 'I-Time', 'I-Time', 'B-Old', 'O', 'O', 'O', 'O'] -> ['O', 'O', 'I-Means', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Time', 'O', 'I-Goal', 'I-Goal', 'O', 'O', 'O'] -> None
srl model: framenet
language: ko
version: 1.2
using viterbi: False
using masking: True
pr

In [117]:
pprint(extracted)

{1: (['신체의',
      '세포,',
      '조직,',
      '장기가',
      '손상되어',
      '더',
      '이상',
      '제',
      '기능을',
      '하지못할',
      '때에',
      '이를',
      '대체하기',
      '위해',
      '이식을',
      '실시한다.'],
     ['O',
      'O',
      'I-Means',
      'O',
      'O',
      'O',
      'O',
      'O',
      'O',
      'I-Time',
      'O',
      'I-Goal',
      'I-Goal',
      'O',
      'O',
      'O'],
     ' '),
 2: (['이때', '이식으로', '옮겨', '붙이는', '세포,', '조직,', '장기를', '이식편이라', '한다.'],
     ['B-Time', 'B-Goal', 'I-Goal', 'O', 'B-Items', 'O', 'O', 'O', 'O'],
     ' '),
 3: (['자신이나',
      '일란성',
      '쌍둥이의',
      '이식편을',
      '이용할',
      '',
      '수',
      '없다면',
      '다른',
      '사람의',
      '이식편으로',
      '‘동종',
      '이식’을',
      '실시한다.'],
     ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
     ' ')}


### TODO : 여기서부터 extracted를 가지고 purpose / goal / effect / entity 등을 찾아야함

In [88]:
for k, tup in extracted.items():
  a, b, c = tup
  roles = [0]*len(b)
  words = [0]*len(b)
  for i in range(len(b)):
    try: 
      roles[i] = b[i].split("-")[1] 
      
    except: 
      roles[i] = b[i].split("-")[0]
  print(roles)
  for _ in roles:
    pass

['Goal', 'Goal', 'O', 'O', 'Means', 'Means', 'Means']


ValueError: ignored

In [21]:
# 1st sentence : GOALS & MEANS 

parsed = parser.parser(ls[0], sent_id='1', result_format='conll')
words = parsed[0][0]
print("😀 Words vector from sentence ",words)
q = [-1]*len(parsed)
for i in range(len(parsed)):
  count=0
  for element in parsed[i][3]:
    if element.startswith("O"):
      count+=1
  q[i] = len(words) - count
hq = heapq.nlargest(2, q)


print("Number of parsed candidates ",len(parsed))
print(q, heapq.nlargest(2, q)) # 첫번째 경우 사용


words = parsed[0][0]
roles = parsed[0][2]
tagged = parsed[0][3]
for idx in range(len(tagged)):
  try:
    roles[i] = tagged[i].split("-")[1]
  except:
    roles[i] = tagged[i].split("-")[0]

print("roles ",roles)

goals = []
means = []
for _ in range(len(words)):
  if roles[_]=="Goal":
    goals.append(words[_])
  if roles[_]=='Means':
    means.append(words[_])

# MEANS
MEANS = ' '.join(means)
pos = h.pos(MEANS)
imp = []
for tup in h.pos(MEANS):
  if tup[1]=="N":
    imp.append(tup[0])
MEANS_TRUNCATED = ' '.join(imp)
print(MEANS_TRUNCATED)

# GOALS
GOAL = ' '.join(goals)
pos = h.pos(GOAL)
imp = []
for tup in h.pos(GOAL):
  if tup[1]=="N":
    imp.append(tup[0])
GOAL_TRUNCATED = ' '.join(imp)
print(GOAL_TRUNCATED)

final_string = MEANS_TRUNCATED + ' -- >' + GOAL_TRUNCATED
print("😀 final string to be inserted",final_string)

[['TEXT2PPTX의', '구현하기', '위해', 'two', 'track', 'process를', '거쳤습니다.'], ['_', '_', '위하다.v', '_', '_', '_', '_'], ['_', '_', 'Purpose', '_', '_', '_', '_'], ['B-Goal', 'I-Goal', 'O', 'O', 'I-Means', 'I-Means', 'I-Means']]


['O', 'O', 'O', 'B-Event', 'I-Event', 'I-Event', 'O']


In [ ]:
# 2nd sentence : PURPOSE & INSTRUMENT
parsed = parser.parser(ls[1], sent_id='1', result_format='conll')
words = parsed[0][0]
print("😀 Words vector from sentence ",words)
q = [-1]*len(parsed)
for i in range(len(parsed)):
  count=0
  for element in parsed[i][3]:
    if element.startswith("O"):
      count+=1
  q[i] = len(words) - count


print("Number of parsed candidates ",len(parsed))
print(q, heapq.nlargest(3, q))
print(itemgetter(*[1,5])(parsed))
candidates = itemgetter(*[1,8])(parsed)
for _ in candidates:
  words = _[0]
  tagged = _[3]
  print("tagged vector of candidates: ",tagged)
  roles = _[2]
  for i in range(len(parsed)):
    try:
      roles[i] = tagged[i].split("-")[1]
    except:
      roles[i] = tagged[i].split("-")[0]
  print(roles,f'Tagged words for the roles : {tagged}')
# roles are selected as the one with purpose and instrument

purpose = []
instrument = []
for _ in range(len(words)):
  if roles[_]=="Purpose":
    purpose.append(words[_])
    
  elif roles[_]=="Instrument":
    instrument.append(words[_])
    
  elif roles[_]!="_": # using 포함
    instrument.append(words[_])
print("Instrument: ",instrument)
print("Purpose: ",purpose)





# PURPOSE
PURPOSE = ' '.join(purpose)
pos = h.pos(PURPOSE)
imp = []
for tup in pos:
  if tup[1]=="N":
    imp.append(tup[0])
PURPOSE_TRUNCATED = ' '.join(imp)
print(PURPOSE_TRUNCATED)

# INSTRUMENT
INSTRUMENT = ' '.join(instrument)
pos = h.pos(INSTRUMENT)
imp = []
for tup in pos:
  if tup[1]=="N":
    imp.append(tup[0])
INST_TRUNCATED = ' '.join(imp)
print(INST_TRUNCATED)

final_string = INST_TRUNCATED+ ' ➡️' + PURPOSE_TRUNCATED 
print("😀 final string to be inserted ", final_string)

    
    

/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/utils.py:279: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


😀 Words vector from sentence  ['대본을', '피피티로', '옮기기', '전에', '요약하고', '분석하기', '위해', '자연어처리의', '최신', '기술을', '다수', '사용하였습니다.']
Number of parsed candidates  9
[2, 11, 4, 0, 6, 1, 1, 1, 11] [11, 11, 6]
([['대본을', '피피티로', '옮기기', '전에', '요약하고', '분석하기', '위해', '자연어처리의', '최신', '기술을', '다수', '사용하였습니다.'], ['_', '_', '_', '전.n', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', 'Time_vector', '_', '_', '_', '_', '_', '_', '_', '_'], ['B-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'O', 'B-Event', 'I-Event', 'I-Event', 'I-Event', 'I-Event', 'I-Event', 'I-Event', 'I-Event']], [['대본을', '피피티로', '옮기기', '전에', '요약하고', '분석하기', '위해', '자연어처리의', '최신', '기술을', '다수', '사용하였습니다.'], ['_', '_', '_', '_', '_', '_', '_', '_', '최신.n', '_', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', 'Relative_time', '_', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Focal_participant', 'O', 'O']])
tagged vector of candidates:  ['B-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'O', 'B-Event

In [ ]:
# 3rd sentence : Goal - Means
parsed = parser.parser(ls[2], sent_id='1', result_format='conll')
words = parsed[0][0]
print("Words vector from sentence ",words)
q = [-1]*len(parsed)
for i in range(len(parsed)):
  count=0
  for element in parsed[i][3]:
    if element.startswith("O"):
      count+=1
  q[i] = len(words) - count

print(q, heapq.nlargest(3, q))

a,b,c=parsed
tagged = b[3]
for i in range(len(tagged)):
  try:
    roles[i] = tagged[i].split("-")[1]
  except:
    roles[i] = tagged[i].split("-")[0]



goals = []
means = []
for _ in range(len(words)):
  if roles[_]=="Goal":
    goals.append(words[_])
  if roles[_]=='Means':
    means.append(words[_])

# MEANS
MEANS = ' '.join(means)
pos = h.pos(MEANS)
imp = []
for tup in h.pos(MEANS):
  if tup[1]=="N":
    imp.append(tup[0])
MEANS_TRUNCATED = ' '.join(imp)
print(MEANS_TRUNCATED)

# GOALS
GOAL = ' '.join(goals)
pos = h.pos(GOAL)
imp = []
for tup in h.pos(GOAL):
  if tup[1]=="N":
    imp.append(tup[0])
GOAL_TRUNCATED = ' '.join(imp)
print(GOAL_TRUNCATED)

final_string = MEANS_TRUNCATED + ' -- >' + GOAL_TRUNCATED
print("😀 final string to be inserted",final_string)


/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/utils.py:279: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


Words vector from sentence  ['또한', '파이썬에서', '파워포인트의', '소스에', '접근하기', '위해', 'xml', '코드를', '심층적으로', '분석했습니다.']
[2, 8, 3] [8, 3, 2]
askdf
askdf
askdf
askdf
   
코드 심층적 분석
파이썬 파워포인트 소스 접근하기
😀 final string to be inserted 코드 심층적 분석-->파이썬 파워포인트 소스 접근하기


In [ ]:
# # Fourth Sentence

# parsed = parser.parser(ls[3], sent_id='1', result_format='conll')
# words = parsed[0][0]
# print("😀 Words vector from sentence ",words)
# q = [-1]*len(parsed)
# for i in range(len(parsed)):
#   count=0
#   for element in parsed[i][3]:
#     if element.startswith("O"):
#       count+=1
#   q[i] = len(words) - count


# print("Number of parsed candidates ",len(parsed))
# print(q, heapq.nlargest(3, q))


candidates = itemgetter(*[0,7])(parsed)
for _ in candidates:
  words = _[0]
  tagged = _[3]
  print("tagged vector of candidates: ",tagged)
  roles = _[2]
  for i in range(len(tagged)):
    try:
      roles[i] = tagged[i].split("-")[1]
    except:
      roles[i] = tagged[i].split("-")[0]

  print(words,f'\nTagged words for the roles : ',roles)
  print()
# roles are selected as the one with effects only

tagged = parsed[0][3]
roles = parsed[0][2]
for i in range(len(tagged)):
    try:
      roles[i] = tagged[i].split("-")[1]
    except:
      roles[i] = tagged[i].split("-")[0]

print(words,'\nTagged words for the roles : ',roles)  
print()



purpose = []
instrument = []
effects = []
entity = []

for _ in range(len(words)):
  if roles[_]=="Purpose":
    purpose.append(words[_])
    
  if roles[_]=="Instrument":
    instrument.append(words[_])
  
  if roles[_]=="Effect":
    effects.append(words[_])
    
  
  if roles[_]=="Entity":
    entity.append(words[_])
    
  elif roles[_]!="_": # using 포함
    instrument.append(words[_])
print("Effects ",effects)


# Effects
EFFECT = ' '.join(effects)
pos = h.pos(EFFECT)
imp = []
for tup in pos:
  if tup[1]=="N":
    imp.append(tup[0])
EFFECT_TRUNCATED = ' '.join(imp)
print(EFFECT_TRUNCATED)

final_string = EFFECT_TRUNCATED
print("😀 final string to be inserted ", final_string)

    
    


tagged vector of candidates:  ['O', 'O', 'B-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect']
['그', '결과', '사용자가', '자연어로', '쓰인', '대본을', '텍2피에', '제공하면', '높은', '수준의', '피피티로', '제공할', '수', '있게', '되었습니다.'] 
Tagged words for the roles :  ['O', 'O', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect']

tagged vector of candidates:  ['O', 'O', 'B-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Event', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'O', 'O']
['그', '결과', '사용자가', '자연어로', '쓰인', '대본을', '텍2피에', '제공하면', '높은', '수준의', '피피티로', '제공할', '수', '있게', '되었습니다.'] 
Tagged words for the roles :  ['O', 'O', 'Entity', 'Entity', 'Entity', 'Entity', 'Entity', 'Entity', 'Event', 'Entity', 'Entity', 'Entity', 'Entity', 'O', 'O']

['그', '결과', '사용자가', '자연어로', '쓰인', '대본을', '텍2피에', '제공하면', '높은', '수준의', '피피티

## PPTX에 활용

In [ ]:
prs = Presentation("final_template.pptx")  # 원하는 template 종류 불러오기
slide_0 = prs.slides[0] # 제복 slide (제목 슬라이드)
slide_2 = prs.slides[2] # 두 번째 slide

In [ ]:
slide_2.placeholders.element[3][2][2][1][1].text = "Two Track Process" # 여덟 번째 슬라이드 제목
slide_2.placeholders.element[5][2][2][1][1].text = ""
slide_2.placeholders.element[6][2][2][1][1].text = ""
slide_2.placeholders.element[17][2][2][1][1].text = ""
slide_2.placeholders.element[18][2][2][1][1].text = ""
slide_2.placeholders.element[19][2][2][1][1].text = "TEXT2PPTX"
slide_2.placeholders.element[11][2][2][1][1].text = "자연어처리의 최신 기술을 다수 사용"
slide_2.placeholders.element[10][2][2][1][1].text = "대본을 피피티로 옮기기 전에 요약하고 분석"
slide_2.placeholders.element[14][2][2][1][1].text = "파이썬에서 파워포인트의 소스에 접근하기 위해 xml 코드를 심층적으로 분석"
slide_2.placeholders.element[13][2][2][1][1].text = "사용자가 자연어로 쓰인 대본을 텍2피에 제공하면 높은 수준의 피피티로 제공"

## 교육용 교재에 활용